In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import pathlib
import scipy.io as sio
from os.path import dirname, join as pjoin
import os
from shutil import copyfile
from sklearn.model_selection import train_test_split
import imageio
from PIL import Image
from random import randint
import scipy

In [4]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [5]:
from whitening import whiten
import matplotlib.image as mpimg
import PIL.Image
import cv2
from skimage import io, color
from resizeimage import resizeimage
from tqdm.auto import tqdm

In [ ]:
print(tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
mirrored_strategy = tf.distribute.MirroredStrategy()

In [ ]:
nombre_image_train = 0 #16417
nombre_image_test = 0 #4162

In [ ]:
if os.path.exists(pathlib.Path().joinpath('Images//n02105855-Shetland_sheepdog//n02105855_2933.jpg')):
    os.remove(pathlib.Path().joinpath('Images//n02105855-Shetland_sheepdog//n02105855_2933.jpg')) 

# Creation des dossiers images du train et du set

In [ ]:
if not os.path.exists('train'):
    os.makedirs('train')
if not os.path.exists('test'):
    os.makedirs('test')


for repertoir in os.listdir('Images'):
    if not os.path.exists('train/'+repertoir):
        os.makedirs('train/'+repertoir)
    if not os.path.exists('test/'+repertoir):
        os.makedirs('test/'+repertoir)

    tmp_data = os.listdir(pathlib.Path().joinpath('Images/'+repertoir).absolute())
    x_train, x_test = train_test_split(tmp_data, test_size = 0.2)

    for img1 in x_train:
        nombre_image_train = nombre_image_train + 1
        copyfile(pathlib.Path().joinpath('Images/'+repertoir+'/'+img1).absolute(), pathlib.Path().joinpath('train/'+repertoir+'/'+img1).absolute())

    for img2 in x_test:
        nombre_image_test = nombre_image_test + 1
        copyfile(pathlib.Path().joinpath('Images/'+repertoir+'/'+img2).absolute(), pathlib.Path().joinpath('test/'+repertoir+'/'+img2).absolute())


In [ ]:
nombre_image_train = 16417
nombre_image_test = 4162

# Traitement des images

## Resize

In [ ]:
directory_train = 'test/' 
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        img = os.path.join(subdir, file)
        with open(img, 'r+b') as f:
            with Image.open(f) as image:
                if(image.size > (150,150,3)):
                    cover = resizeimage.resize_cover(image, (150, 150))
                    cover.save(img, image.format)
                pbar.update(1/nombre_image_test)
pbar.close()
                
directory_train = 'train/'  
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        img = os.path.join(subdir, file)
        with open(img, 'r+b') as f:
            with Image.open(f) as image:
                if(image.size > (150,150,3)):
                    cover = resizeimage.resize_cover(image, (150, 150))
                    cover.save(img, image.format)
                pbar.update(1/nombre_image_train)
pbar.close()

## Whitening

In [ ]:
def whitening(img):
    image = PIL.Image.open(img)
    foreground, background = whiten(image, kernel_size=20, downsample=4)
    foreground.save(img, 'jpeg')
                                    
# image = plt.show('Images/n02085620-Chihuahua/'+tmp_data[0])
# foreground, background = whiten(image, kernel_size=20, downsample=4)
# foreground.save('foreground.jpg', 'jpeg')

# On s'occupe des images dans train  
directory_train = 'train/' 
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        a = os.path.join(subdir, file)
        whitening(a)
        pbar.update(1/nombre_image_train)
pbar.close()
    
# On s'occupe des images dans test
directory_test= 'test/'
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_test):
    for file in files:
        a = os.path.join(subdir, file)
        whitening(a)
        pbar.update(1/nombre_image_test)
pbar.close()

## Equalization

In [ ]:
def equalization(image):
    img = cv2.imread(image)
    img_to_yuv = cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
    img_to_yuv[:,:,0] = cv2.equalizeHist(img_to_yuv[:,:,0])
    hist_equalization_result = cv2.cvtColor(img_to_yuv, cv2.COLOR_YUV2BGR)
    cv2.imwrite(image,hist_equalization_result)
    
    
directory_train = 'train/'
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        a = os.path.join(subdir, file)
        equalization(a)
        pbar.update(1/nombre_image_train)
pbar.close()

directory_train = 'test/'
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        a = os.path.join(subdir, file)
        equalization(a)
        pbar.update(1/nombre_image_test)
pbar.close()

# Augmention du train

In [ ]:
def reName(imgName, endString, idString):
    index = imgName.find(endString)
    if index != -1 :
        newImageName = imgName[0:(index-1)]
        newImageName = newImageName + idString + endString
        return newImageName
    else:
        return imgName

## Mirroring

In [ ]:
def mirror(img):
    image = cv2.imread(img)
    mirrorImage = cv2.flip(image, 1)
    newImgName = reName(img, '.jpg', '_m')
    mirrorImage = Image.fromarray(mirrorImage)
    mirrorImage.save(newImgName , 'jpeg')

## Cropping

In [ ]:
def zoom(img):
    image = cv2.imread(img)
    h, w = image.shape[:2]
    zoomImg = image[h//8 : -h//8,w//8 : -w//8]
    newImgName = reName(img, '.jpg', '_z')
    zoomImg = Image.fromarray(zoomImg)
    zoomImg.save(newImgName , 'jpeg')

## Rotate

In [ ]:
def rotate(img):
    value = randint(-90, 90)
    image = Image.open(img)
    image = image.rotate(value)
    newImgName = reName(img, '.jpg', '_r')
    image.save(newImgName , 'jpeg')

In [ ]:
# On s'occupe des images dans train    
directory_train = 'train/' 
pbar = tqdm(total=1)
for subdir, dirs, files in os.walk(directory_train):
    for file in files:
        if file.find('_r') != -1 or file.find('_z') != -1 or file.find('_m') != -1:
            continue
        else:
            a = os.path.join(subdir, file)
            rotate(a)
            #zoom(a)
            mirror(a)
            pbar.update(1/nombre_image_train)
pbar.close()

# Recuperation du train et du set

In [6]:
training_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [15]:
train = training_data.flow_from_directory(
    pathlib.Path().joinpath('Train').absolute(),
    target_size=(150,150),
    class_mode='categorical'
)

test = training_data.flow_from_directory(
    pathlib.Path().joinpath('test').absolute(),
    target_size=(150,150),
    class_mode='categorical'
)

Found 4355 images belonging to 10 classes.
Found 388 images belonging to 10 classes.


# Model

In [19]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(500, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [20]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Entrainement du model

In [ ]:
history = model.fit(train,
                    epochs=100,
                   validation_data=train, 
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
                               tfdocs.modeling.EpochDots(),
                              ])

Epoch 1/100
137/137 [==============================] - ETA: 0s - loss: 2.4734 - accuracy: 0.1437
Epoch: 0, accuracy:0.1437,  loss:2.4734,  val_accuracy:0.2142,  val_loss:2.2047,  
137/137 [==============================] - 118s 864ms/step - loss: 2.4734 - accuracy: 0.1437 - val_loss: 2.2047 - val_accuracy: 0.2142
Epoch 2/100
137/137 [==============================] - 117s 854ms/step - loss: 2.2013 - accuracy: 0.2135 - val_loss: 2.0879 - val_accuracy: 0.2661
Epoch 3/100
137/137 [==============================] - 117s 854ms/step - loss: 2.1297 - accuracy: 0.2445 - val_loss: 2.0361 - val_accuracy: 0.2799
Epoch 4/100
137/137 [==============================] - 117s 854ms/step - loss: 2.0758 - accuracy: 0.2677 - val_loss: 1.9950 - val_accuracy: 0.3118
Epoch 5/100
137/137 [==============================] - 117s 853ms/step - loss: 2.0262 - accuracy: 0.2983 - val_loss: 1.9572 - val_accuracy: 0.3235
Epoch 6/100
137/137 [==============================] - 117s 853ms/step - loss: 1.9679 - accuracy:

# Sauvegarde du model

In [ ]:
model.save('snooky.h5')

# Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(test)

# Load du  Model

In [ ]:
model = tf.keras.models.load_model('snooky.h5')

# Courbe d'apprentissage

In [ ]:
# summarize history for accuracy
plt.plot(history.history['mae'], label="train")
plt.plot(history.history['val_mae'], label="test")
plt.title('mean absolute error')
plt.ylim([0,10])
plt.ylabel('MAE [MPG]')
plt.xlabel('epoch')
plt.legend()
plt.show()
# summarize history for loss
plt.plot(history.history['mse'], label="train")
plt.plot(history.history['val_mse'], label="test")
plt.title('mean squarre error')
plt.ylabel('MSE [MPG^2]')
plt.xlabel('epoch')
plt.ylim([0,20])
plt.legend()
plt.show()

# Prediction

In [ ]:
class_names = []
for repertoir in os.listdir('Images'):
    class_names.append(repertoire)
    
img = mpimg.imread("monimage.png")

prediction = model.predict(img)

plt.bar(range(120),predictions[0])
plt.xticks(range(120), class_names, rotation=45)